In [1]:
!pip install -qqq openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 328.5/328.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00


In [4]:
import os
from google.colab import userdata
os.environ['OPENAI_API_KEY'] = userdata.get('openai-key')

## Zero-shot categorization

In [18]:
import random

# File path
file_path = 'Sentences_AllAgree.txt'

# Function to load and categorize news titles
def load_and_categorize_news(file_path):
    positive = []
    negative = []
    neutral = []

    with open(file_path, 'r', encoding='utf-8', errors='ignore') as file:
        for line in file:
            if line.strip().endswith('@positive'):
                positive.append(line.strip())
            elif line.strip().endswith('@negative'):
                negative.append(line.strip())
            elif line.strip().endswith('@neutral'):
                neutral.append(line.strip())

    return positive, negative, neutral

# Function to randomly extract 20 titles from each category
def get_random_titles(positive, negative, neutral, num_samples):
    selected_positive = random.sample(positive, num_samples)
    selected_negative = random.sample(negative, num_samples)
    selected_neutral = random.sample(neutral, num_samples)

    return selected_positive, selected_negative, selected_neutral

# Load and categorize news titles
positive, negative, neutral = load_and_categorize_news(file_path)

print(f"Positive titles: {len(positive)}")
print(positive[0])
print(f"Negative titles: {len(negative)}")
print(f"Neutral titles: {len(neutral)}")

# Get random titles
selected_positive, selected_negative, selected_neutral = get_random_titles(positive, negative, neutral, 20)

# Combine titles in the required order
ordered_titles = selected_positive + selected_negative + selected_neutral

# Function to strip labels from titles and keep labels
def strip_labels(titles):
    stripped_titles = []
    labels = []
    for title in titles:
        label = title.split('@')[-1]
        stripped_title = title.rsplit('@', 1)[0]
        stripped_titles.append(stripped_title)
        labels.append(label)
    return stripped_titles, labels

# Strip labels from titles and keep the labels
stripped_titles, original_labels = strip_labels(ordered_titles)
print(f"Stripped titles: {len(stripped_titles)}")
print(stripped_titles[0])
print(f"Original labels: {len(original_labels)}")
print(original_labels[0])


Positive titles: 570
For the last quarter of 2010 , Componenta 's net sales doubled to EUR131m from EUR76m for the same period a year earlier , while it moved to a zero pre-tax profit from a pre-tax loss of EUR7m .@positive
Negative titles: 303
Neutral titles: 1391
Stripped titles: 60
Equity ratio was 60.9 % compared to 54.2 % In the third quarter of 2007 , net sales of the Frozen Foods Business totaled EUR 11.0 , up by about 5 % from the third quarter of 2006 .
Original labels: 60
positive


##Prepare the chat-completion API for zero-shot categorization

In [19]:
from openai import OpenAI
client = OpenAI()

# Define the zero-shot prompt
def create_prompt(title):
    return f"Please categorize the following news title as Positive, Negative, or Neutral on the stock price from the viewpoint of a retail investor.:\n\nTitle: {title}"

# Function to categorize a title using OpenAI's API
def categorize_title(title):
    prompt = create_prompt(title)
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that categorizes news titles correctly."},
            {"role": "user", "content": prompt}
        ]
    )
    # return response.choices[0].message.content.strip()
    # Ensure only the label is returned
    response_text = response.choices[0].message.content.strip().lower()
    if "positive" in response_text:
        return "positive"
    elif "negative" in response_text:
        return "negative"
    elif "neutral" in response_text:
        return "neutral"
    else:
        return "unknown" # handle unexpected cases

# Categorize each title and store the results
predicted_labels = [categorize_title(title) for title in stripped_titles]

# Convert original labels to lowercase for comparison
original_labels = [label.lower() for label in original_labels]

# Compare the original and predicted labels and calculate accuracy
correct_predictions = sum(1 for orig, pred in zip(original_labels, predicted_labels) if orig == pred)
total_predictions = len(original_labels)
accuracy = correct_predictions / total_predictions

# Print the results and accuracy
print(f"Correct Predictions: {correct_predictions}")
print(f"Accuracy: {accuracy * 100:.2f}%")

# Print detailed results
for title, orig_label, pred_label in zip(stripped_titles, original_labels, predicted_labels):
    print(f"Title: {title}\nOriginal Label: {orig_label}\nPredicted Label: {pred_label}\n")


Correct Predictions: 50
Accuracy: 83.33%
Title: Equity ratio was 60.9 % compared to 54.2 % In the third quarter of 2007 , net sales of the Frozen Foods Business totaled EUR 11.0 , up by about 5 % from the third quarter of 2006 .
Original Label: positive
Predicted Label: neutral

Title: This location makes the mall a convenient place to shop for consumers from three large residential areas nearby : Jaroszowka , Wysockiego and Zgody .
Original Label: positive
Predicted Label: neutral

Title: The company 's market share is continued to increase further .
Original Label: positive
Predicted Label: positive

Title: Finnish Sampo Bank , of Danish Danske Bank group , reports profit before taxes of EUR 152.3 mn in 2010 , up from EUR 32.7 mn in 2009 .
Original Label: positive
Predicted Label: positive

Title: In August-October 2010 , the company 's result before taxes totalled EUR 9.6 mn , up from EUR 0.5 mn in the corresponding period in 2009 .
Original Label: positive
Predicted Label: positive

##**RESULTS & OBSERVATIONS**

As seen above, the zero-shot prompting for 60 news titles, resulted in the accuracy of 83.30%, predicting sentiment of 50 news titles accurately.

##12 Shots Learning

In [20]:
# Define example news titles with corresponding sentiment labels
examples = [
    {"title": "According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .", "label": "neutral"},
    {"title": "With the new production plant the company would increase its capacity to meet the expected increase in demand and would improve the use of raw materials and therefore increase the production profitability .", "label": "positive"},
    {"title": "For the last quarter of 2010 , Componenta 's net sales doubled to EUR131m from EUR76m for the same period a year earlier , while it moved to a zero pre-tax profit from a pre-tax loss of EUR7m .", "label": "positive"},
    {"title": "In the third quarter of 2010 , net sales increased by 5.2 % to EUR 205.5 mn , and operating profit by 34.9 % to EUR 23.5 mn .", "label": "positive"},
    {"title": "Operating profit rose to EUR 13.1 mn from EUR 8.7 mn in the corresponding period in 2007 representing 7.7 % of net sales .", "label": "positive"},
    {"title": "Operating profit totalled EUR 21.1 mn , up from EUR 18.6 mn in 2007 , representing 9.7 % of net sales .", "label": "positive"},
    {"title": "TeliaSonera TLSN said the offer is in line with its strategy to increase its ownership in core business holdings and would strengthen Eesti Telekom 's offering to its customers .", "label": "positive"},
    {"title": "STORA ENSO , NORSKE SKOG , M-REAL , UPM-KYMMENE Credit Suisse First Boston ( CFSB ) raised the fair value for shares in four of the largest Nordic forestry groups .", "label": "positive"},
    {"title": "A purchase agreement for 7,200 tons of gasoline with delivery at the Hamina terminal , Finland , was signed with Neste Oil OYj at the average Platts index for this September plus eight US dollars per month .", "label": "positive"},
    {"title": "Finnish Talentum reports its operating profit increased to EUR 20.5 mn in 2005 from EUR 9.3 mn in 2004 , and net sales totaled EUR 103.3 mn , up from EUR 96.4 mn .", "label": "positive"},
    {"title": "Clothing retail chain Sepp+�l+� 's sales increased by 8 % to EUR 155.2 mn , and operating profit rose to EUR 31.1 mn from EUR 17.1 mn in 2004 .", "label": "positive"},
    {"title": "Consolidated net sales increased 16 % to reach EUR74 .8 m , while operating profit amounted to EUR0 .9 m compared to a loss of EUR0 .7 m in the prior year period .", "label": "positive"}
]


# Define the zero-shot prompt with examples
def create_prompt_with_examples(title):
    examples_text = "\n".join([f"Title: {ex['title']}\nLabel: {ex['label']}" for ex in examples])
    return f"Here are some examples of news titles with their corresponding sentiment labels:\n\n{examples_text}\n\nBased on these examples, please categorize the following news title as Positive, Negative, or Neutral:\n\nTitle: {title}"

# Function to categorize a title using OpenAI's API with examples
def categorize_title_with_examples(title):
    prompt = create_prompt_with_examples(title)
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are a helpful assistant that labels sentiments of news titles as in the examples provided."},
            {"role": "user", "content": prompt}
        ]
    )
    # Ensure only the label is returned
    response_text = response.choices[0].message.content.strip().lower()
    if "positive" in response_text:
        return "positive"
    elif "negative" in response_text:
        return "negative"
    elif "neutral" in response_text:
        return "neutral"
    else:
        return "unknown"  # Handle unexpected cases

# Categorize each title and store the results using the new function with examples
predicted_labels_with_examples = [categorize_title_with_examples(title) for title in stripped_titles]

# Compare the original and predicted labels and calculate accuracy
correct_predictions_with_examples = sum(1 for orig, pred in zip(original_labels, predicted_labels_with_examples) if orig == pred)
total_predictions_with_examples = len(original_labels)
accuracy_with_examples = correct_predictions_with_examples / total_predictions_with_examples

# Print the results and accuracy
print(f"Correct Predictions with Examples: {correct_predictions_with_examples}")
print(f"Accuracy with Examples: {accuracy_with_examples * 100:.2f}%")

# Print detailed results with examples
for title, orig_label, pred_label in zip(stripped_titles, original_labels, predicted_labels_with_examples):
    print(f"Title: {title}\nOriginal Label: {orig_label}\nPredicted Label with Examples: {pred_label}\n")


Correct Predictions with Examples: 55
Accuracy with Examples: 91.67%
Title: Equity ratio was 60.9 % compared to 54.2 % In the third quarter of 2007 , net sales of the Frozen Foods Business totaled EUR 11.0 , up by about 5 % from the third quarter of 2006 .
Original Label: positive
Predicted Label with Examples: positive

Title: This location makes the mall a convenient place to shop for consumers from three large residential areas nearby : Jaroszowka , Wysockiego and Zgody .
Original Label: positive
Predicted Label with Examples: positive

Title: The company 's market share is continued to increase further .
Original Label: positive
Predicted Label with Examples: positive

Title: Finnish Sampo Bank , of Danish Danske Bank group , reports profit before taxes of EUR 152.3 mn in 2010 , up from EUR 32.7 mn in 2009 .
Original Label: positive
Predicted Label with Examples: positive

Title: In August-October 2010 , the company 's result before taxes totalled EUR 9.6 mn , up from EUR 0.5 mn in

##**RESULTS & OBSERVATIONS**

After providing just 12 examples of the correct sentiments fo, the accuracy improved to 91.67% for the same data set of news titles as used earlier. It is evident that with 12-shot prompting, there is an uptick in accuracy.

##Create Training and Validation Datasets

In [42]:
import pandas as pd
import json

# Function to load data from a file
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8', errors='replace') as file:
        lines = file.readlines()
    data = []
    for line in lines:
        sentence, sentiment = line.rsplit('@', 1)
        data.append({'sentence': sentence.strip(), 'sentiment': sentiment.strip()})
    return pd.DataFrame(data)

# Load datasets
df_75 = load_data('/content/Sentences_75Agree.txt')
df_all = load_data('/content/Sentences_AllAgree.txt')

# Combine the datasets
combined_data = pd.concat([df_75, df_all]).reset_index(drop=True)

# Function to sample data with a check for available samples
def sample_data(df, sentiment, n):
    available = len(df[df['sentiment'] == sentiment])
    if available < n:
        print(f"Warning: Only {available} samples available for sentiment '{sentiment}'")
        return df[df['sentiment'] == sentiment]
    return df[df['sentiment'] == sentiment].sample(n, random_state=42)

# Sample 75 items for each sentiment from Sentences_75Agree.txt
train_neutral_75 = sample_data(df_75, 'neutral', 75)
train_positive_75 = sample_data(df_75, 'positive', 75)
train_negative_75 = sample_data(df_75, 'negative', 75)

# Sample 75 items for each sentiment from Sentences_AllAgree.txt
train_neutral_all = sample_data(df_all, 'neutral', 75)
train_positive_all = sample_data(df_all, 'positive', 75)
train_negative_all = sample_data(df_all, 'negative', 75)

# Combine to create a total of 150 samples for each sentiment from both files
training_data = pd.concat([train_neutral_75, train_positive_75, train_negative_75,
                           train_neutral_all, train_positive_all, train_negative_all],
                          ignore_index=True)

# Ensure the test dataset does not include any training samples
remaining_data = combined_data[~combined_data['sentence'].isin(training_data['sentence'])]

# Create the test dataset by sampling 500 unique entries
test_data = remaining_data.sample(500, random_state=42).reset_index(drop=True)

# Function to convert DataFrame to chat JSONL format
def convert_to_chat_jsonl(df, file_path):
    with open(file_path, 'w', encoding='utf-8') as f:
        for _, row in df.iterrows():
            record = {
                "messages": [
                    {"role": "system", "content": "You are a helpful assistant to determine the sentiment of a news sentence."},
                    {"role": "user", "content": row['sentence']},
                    {"role": "assistant", "content": row['sentiment']}
                ]
            }
            f.write(json.dumps(record) + '\n')

# Save training data to chat JSONL format
convert_to_chat_jsonl(training_data, '/content/training_data.jsonl')

# Save test data to chat JSONL format
convert_to_chat_jsonl(test_data, '/content/test_data.jsonl')

print("Training and test data have been created and saved in chat JSONL format.")


Training and test data have been created and saved in chat JSONL format.


In [43]:
from openai import OpenAI
client = OpenAI()

##Creating file ID

In [44]:
with open('/content/training_data.jsonl', 'rb') as file:
    result = client.files.create(
        file=file,
        purpose='fine-tune'
    )

# Get the file ID
file_id = result.id

print(f"File ID: {file_id}")

File ID: file-m2XUii73IdOwoANGZeb7PcnQ


In [45]:

client.fine_tuning.jobs.create(
  training_file=file_id,
  model="gpt-3.5-turbo"
)

FineTuningJob(id='ftjob-Hl6mcJmb6Gbim4j3GPkUtgM2', created_at=1720738446, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-8fO0aTFH5mkX8nA1hFzrrdwm', result_files=[], seed=512520200, status='validating_files', trained_tokens=None, training_file='file-m2XUii73IdOwoANGZeb7PcnQ', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [46]:

client.fine_tuning.jobs.list()

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-Hl6mcJmb6Gbim4j3GPkUtgM2', created_at=1720738446, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-8fO0aTFH5mkX8nA1hFzrrdwm', result_files=[], seed=512520200, status='validating_files', trained_tokens=None, training_file='file-m2XUii73IdOwoANGZeb7PcnQ', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None), FineTuningJob(id='ftjob-l0zH9GQNcFMEbSomySe6rD5O', created_at=1720733835, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal::9jwQ7Gul', finished_at=1720736354, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-8fO0aT

In [48]:
client.fine_tuning.jobs.retrieve('ftjob-Hl6mcJmb6Gbim4j3GPkUtgM2')

FineTuningJob(id='ftjob-Hl6mcJmb6Gbim4j3GPkUtgM2', created_at=1720738446, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-8fO0aTFH5mkX8nA1hFzrrdwm', result_files=[], seed=512520200, status='validating_files', trained_tokens=None, training_file='file-m2XUii73IdOwoANGZeb7PcnQ', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

##Validate the Pre-trained Model

In [7]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report
import openai
import numpy as np


# Function to get completion from the model
client = openai.OpenAI()

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

# Load the test data
test_data = pd.read_json('/content/test_data.jsonl', lines=True)
test_data['sentence'] = test_data['messages'].apply(lambda x: x[1]['content'])
test_data['sentiment'] = test_data['messages'].apply(lambda x: x[2]['content'])

# Function to get predictions from the pre-trained model
def get_predictions(test_data):
    predictions = []
    for index, row in test_data.iterrows():
        prompt = f"Determine the sentiment of this news sentence: {row['sentence']}"
        actual_label = row['sentiment']
        predicted_label = get_completion(prompt, model="ft:gpt-3.5-turbo-0125:personal::9jxq0o9A")
        predictions.append((actual_label, predicted_label))
    return predictions

# Get predictions
predictions = get_predictions(test_data)

# Extract actual and predicted labels
actual_labels = [label for label, _ in predictions]
predicted_labels = [label for _, label in predictions]

# Calculate confusion matrix
conf_matrix = confusion_matrix(actual_labels, predicted_labels, labels=['positive', 'negative', 'neutral'])
class_report = classification_report(actual_labels, predicted_labels, labels=['positive', 'negative', 'neutral'])

print("Confusion Matrix:")
print(conf_matrix)
print("\nClassification Report:")
print(class_report)

# Detailed Confusion Matrix with TP, TN, FP, FN
def detailed_confusion_matrix(actual_labels, predicted_labels):
    # Calculate confusion matrix
    cm = confusion_matrix(actual_labels, predicted_labels, labels=['positive', 'negative', 'neutral'])

    # True Positives, False Positives, True Negatives, False Negatives
    tp = np.diag(cm)
    fp = cm.sum(axis=0) - tp
    fn = cm.sum(axis=1) - tp
    tn = cm.sum() - (tp + fp + fn)

    # Create a DataFrame for better visualization
    cm_df = pd.DataFrame({
        'TP': tp,
        'FP': fp,
        'FN': fn,
        'TN': tn
    }, index=['positive', 'negative', 'neutral'])

    return cm_df, cm

# Get detailed confusion matrix
detailed_cm, cm = detailed_confusion_matrix(actual_labels, predicted_labels)
print("\nDetailed Confusion Matrix:")
print(detailed_cm)

# Calculate percentage accuracy
accuracy = (cm.trace() / cm.sum()) * 100
print(f"\nPercentage Accuracy: {accuracy:.2f}%")


Confusion Matrix:
[[122   0   5]
 [  0  47   0]
 [ 16   8 302]]

Classification Report:
              precision    recall  f1-score   support

    positive       0.88      0.96      0.92       127
    negative       0.85      1.00      0.92        47
     neutral       0.98      0.93      0.95       326

    accuracy                           0.94       500
   macro avg       0.91      0.96      0.93       500
weighted avg       0.95      0.94      0.94       500


Detailed Confusion Matrix:
           TP  FP  FN   TN
positive  122  16   5  357
negative   47   8   0  445
neutral   302   5  24  169

Percentage Accuracy: 94.20%


##Pre Trained Model Testing on Real Data

In [15]:


# Define the news articles
news_articles = [
    "US stocks rolled over on Thursday as investors rotated out of tech after a key inflation report showed consumer prices unexpectedly fell on a month-over-month basis for the first time since 2020.",
    "America’s banks are more exposed to a downturn than they appear.",
    "On average, analysts tracking Tesla stock are expecting a slight (2%) year-over-year decline in revenue to just over $24 billion. The bottom-line slump should be more dramatic, as those prognosticators are modeling a collective $0.61 per share for profitability.",
    "Oil rose for a third day as optimism in wider financial markets, spurred by indications that US interest-rate cuts are on the way, helped to propel futures higher.",
    "Costco stock has a 12% upside after raising its annual membership fees for the first time since 2017, according to Bank of America.",
    "Arguably the most significant news to come from the latest consumer price index reading was a pullback in housing-related inflation. Shelter costs rose just 0.2% for the slowest monthly increase in three years.",
    "Intuit (INTU) announced plans to layoff 1,800 employees and close two locations as part of a restructuring plan that will enable the financial technology company to increase its investments in artificial intelligence.",
    "According to early data from market research firm IDC, the PC market grew 3%, marking the second quarter of growth after a staggering seven consecutive quarters of declines.",
    "Germany has ordered a staggered ban on Chinese components in domestic 5G networks by 2029, in a compromise decision that seeks to end a long-running dispute in Berlin over the danger of Beijing’s involvement in critical digital infrastructure.",
    "Delta Air Lines (DAL) issued a downbeat earnings guidance for the ongoing quarter on Thursday as the airline's earnings during the three months to June fell short of market expectations.",
    "The color of the flower is black"
]

# Function to do the classification using the fine tuned model
def classify(prompt, model):
    messages = [
        {"role": "system", "content": "You are a helpful assistant that classifies the sentiment of news articles."},
        {"role": "user", "content": prompt}
    ]
    response = client.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0
    )
    return response.choices[0].message.content

# Classify each news article
model_id = "ft:gpt-3.5-turbo-0125:personal::9jxq0o9A"
for article in news_articles:
    sentiment = classify(article, model_id)
    print(f"Article: {article}\nSentiment: {sentiment}\n")


Article: US stocks rolled over on Thursday as investors rotated out of tech after a key inflation report showed consumer prices unexpectedly fell on a month-over-month basis for the first time since 2020.
Sentiment: negative

Article: America’s banks are more exposed to a downturn than they appear.
Sentiment: negative

Article: On average, analysts tracking Tesla stock are expecting a slight (2%) year-over-year decline in revenue to just over $24 billion. The bottom-line slump should be more dramatic, as those prognosticators are modeling a collective $0.61 per share for profitability.
Sentiment: negative

Article: Oil rose for a third day as optimism in wider financial markets, spurred by indications that US interest-rate cuts are on the way, helped to propel futures higher.
Sentiment: positive

Article: Costco stock has a 12% upside after raising its annual membership fees for the first time since 2017, according to Bank of America.
Sentiment: positive

Article: Arguably the most sig

##Human Level Accuracy Check

What I observed that it, our fine tuned model classified all the 11 given news articles as expected which is showing 100% accuracy of it. Even I gave one sentence "The color of the flower is black" which is not  related to finance and it correctly classified it as Neutral.

##Comparison between Zero Shot, 12 Shhot nad Fine Tuning Model
The performance got improved from **83.30%** to **91.67%** to **94.20%**